### Building a Context-Aware Conversational Agent

#### Overview
This guide walks through building a conversational agent that maintains context across interactions. Using a modern AI framework, we'll design an agent capable of engaging in natural, coherent dialogues.

#### Motivation
Many basic chatbots struggle to maintain context, resulting in disjointed, frustrating experiences for users. This tutorial addresses that challenge by creating an agent that remembers and references previous parts of the conversation, enhancing interaction quality.

#### Key Components
- **Language Model**: The AI core that generates responses.
- **Prompt Template**: Defines the conversation structure.
- **History Manager**: Handles conversation history and context.
- **Message Store**: Saves messages for each session.

#### Method Details

1. **Setting Up the Environment**  
   Start by configuring the required AI framework and securing access to a suitable language model. This setup forms the foundation of the conversational agent.

2. **Creating the Chat History Store**  
   Develop a system to manage multiple sessions, with each session linked to its unique message history.

3. **Defining the Conversation Structure**  
   Build a template that includes:
   - A system message specifying the AI's role
   - A placeholder for conversation history
   - The user's input

   This format guides the AI responses and maintains conversation consistency.

4. **Building the Conversational Chain**  
   Integrate the prompt template with the language model to create a foundational conversational chain. Add a history management component that seamlessly manages inserting and retrieving conversation history.

5. **Interacting with the Agent**  
   Invoke the agent using a user input and session identifier. The history manager retrieves the relevant conversation history, inserts it into the prompt, and stores new messages after each interaction.

#### Conclusion
This conversational agent design offers multiple benefits:
- **Context Awareness**: Enables the agent to reference previous conversation parts, fostering natural interactions.
- **Simplicity**: A modular structure keeps implementation clear and straightforward.
- **Flexibility**: The conversation structure can be easily adjusted, or a new language model integrated.
- **Scalability**: The session-based approach allows multiple independent conversations to be managed.

With this framework, you can further enhance the agent by:
- Utilizing advanced prompt engineering techniques
- Integrating external knowledge sources
- Adding domain-specific capabilities
- Incorporating error handling and conversation repair

By emphasizing context management, this approach significantly enhances traditional chatbot functionality, making way for more engaging and supportive AI-driven interactions.

#### Conversational Agent Tutorial
This notebook demonstrates how to create a simple context-aware conversational agent using LangChain.

In [1]:
%pip install -q langchain langchain_experimental openai python-dotenv langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv

In [5]:
# Load environment variables
load_dotenv() ##load all the nevironment variables
# I have done with my github token if you dont have access to github models you can setup your openai key
token=os.environ['GITHUB_TOKEN']
endpoint = 'https://models.inference.ai.azure.com'

llm = ChatOpenAI(
    openai_api_base=endpoint,
    openai_api_key=token,
    model="gpt-4o-mini",  # Replace with the specific model if required
   
)

In [6]:

store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [8]:

chain = prompt | llm

In [9]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [10]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)

AI: Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
AI: Your previous message was: "Hello! How are you?"
